In [7]:
#EU Capitals Weather Data Collection

import json
import requests
import time
from datetime import datetime

eu_capitals = [
    {"city": "Vienna", "country": "Austria", "lat": 48.2082, "lon": 16.3738},
    {"city": "Brussels", "country": "Belgium", "lat": 50.8503, "lon": 4.3517},
    {"city": "Sofia", "country": "Bulgaria", "lat": 42.6977, "lon": 23.3219},
    {"city": "Zagreb", "country": "Croatia", "lat": 45.8150, "lon": 15.9819},
    {"city": "Nicosia", "country": "Cyprus", "lat": 35.1856, "lon": 33.3823},
    {"city": "Prague", "country": "Czechia", "lat": 50.0755, "lon": 14.4378},
    {"city": "Copenhagen", "country": "Denmark", "lat": 55.6761, "lon": 12.5683},
    {"city": "Tallinn", "country": "Estonia", "lat": 59.4370, "lon": 24.7536},
    {"city": "Helsinki", "country": "Finland", "lat": 60.1695, "lon": 24.9354},
    {"city": "Paris", "country": "France", "lat": 48.8566, "lon": 2.3522},
    {"city": "Berlin", "country": "Germany", "lat": 52.5200, "lon": 13.4050},
    {"city": "Athens", "country": "Greece", "lat": 37.9838, "lon": 23.7275},
    {"city": "Budapest", "country": "Hungary", "lat": 47.4979, "lon": 19.0402},
    {"city": "Dublin", "country": "Ireland", "lat": 53.3498, "lon": -6.2603},
    {"city": "Rome", "country": "Italy", "lat": 41.9028, "lon": 12.4964},
    {"city": "Riga", "country": "Latvia", "lat": 56.9496, "lon": 24.1052},
    {"city": "Vilnius", "country": "Lithuania", "lat": 54.6872, "lon": 25.2797},
    {"city": "Luxembourg", "country": "Luxembourg", "lat": 49.6116, "lon": 6.1319},
    {"city": "Valletta", "country": "Malta", "lat": 35.8989, "lon": 14.5146},
    {"city": "Amsterdam", "country": "Netherlands", "lat": 52.3676, "lon": 4.9041},
    {"city": "Warsaw", "country": "Poland", "lat": 52.2297, "lon": 21.0122},
    {"city": "Lisbon", "country": "Portugal", "lat": 38.7223, "lon": -9.1393},
    {"city": "Bucharest", "country": "Romania", "lat": 44.4268, "lon": 26.1025},
    {"city": "Bratislava", "country": "Slovakia", "lat": 48.1486, "lon": 17.1077},
    {"city": "Ljubljana", "country": "Slovenia", "lat": 46.0569, "lon": 14.5058},
    {"city": "Madrid", "country": "Spain", "lat": 40.4168, "lon": -3.7038},
    {"city": "Stockholm", "country": "Sweden", "lat": 59.3293, "lon": 18.0686}
]

weather_codes = {
    0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
    45: "Fog", 48: "Depositing rime fog",
    51: "Light drizzle", 53: "Moderate drizzle", 55: "Dense drizzle",
    56: "Light freezing drizzle", 57: "Dense freezing drizzle",
    61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
    66: "Light freezing rain", 67: "Heavy freezing rain",
    71: "Slight snow fall", 73: "Moderate snow fall", 75: "Heavy snow fall",
    77: "Snow grains",
    80: "Slight rain showers", 81: "Moderate rain showers", 82: "Violent rain showers",
    85: "Slight snow showers", 86: "Heavy snow showers",
    95: "Thunderstorm",
    96: "Thunderstorm with slight hail", 99: "Thunderstorm with heavy hail"  
}

#Fetch weather data from Open-Meteo API
def fetch_weather(lat, lon):
    try:
        url = 'https://api.open-meteo.com/v1/forecast'
        params = {
            "latitude": lat,
            "longitude": lon,
            "current": "temperature_2m,wind_speed_10m,weather_code", #Curent data that we want
            "hourly": "temperature_2m,precipitation_probability,weather_code", #What hourly data we want
            "timezone": "auto", #Automatically detected timezone
            "forecast_days": 1  #Get data only for today
        }
        response = requests.get(url, params = params, timeout = 10)
        response.raise_for_status() #Check for the HTTP errors
        #Convert the response from JSON format to a Python dictionary
        return response.json()
        
    #Errors
    except requests.exceptions.Timeout:
        print("Request timed out after 10 seconds.")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the API request: {e}")
    except requests.exceptions.ConnectionError:
        print("Failed to connect to the Open-Meteo API server.")
    except KeyError as e:
        print(f"Error parsing weather data: Missing key {e}. Check API response structure.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return None

#Function that takes city ifo and raw API data
def process_city_weather(city_info, raw_data):
    if not raw_data or "current" not in raw_data:
        #Return None to indicate processing failure
        return None
    try:
        current = raw_data["current"] #Get the current weather data from the raw API response
        weather_code = current.get("weather_code", 0)
        current_weather = {
            "temperature": current.get("temperature_2m", 0),
            "windspeed": current.get("wind_speed_10m", 0),
            "weathercode": weather_code,
            "condition": weather_codes.get(weather_code, "Unknown"), #If code is not in the dictionary it will be "Unknown"
            "time": current.get("time", datetime.now().isoformat())
        }
        hourly_forecast = [] #Create empty list to store hourly forecast data
        hourly_data = raw_data.get("hourly", {})
        times = hourly_data.get("time", [])
        temps = hourly_data.get("temperature_2m", [])
        codes = hourly_data.get("weather_code", [])
        precip = hourly_data.get("precipitation_probability", [])

        #Get all available hours (should be 24 for current day)
        #Use min() to ensure we don't try to access more hours than available
        #len(times) gives number of hours, and we limit to maximum 24
        for i in range(min(len(times), 24)):
            #Create a dictionary for this hour's forecast
            hour_forecast = {
                "time": times[i],
                "temperature": temps[i] if i <len(temps) else 0,
                "precipitation_probability": precip[i] if i<len(precip) else 0,
                "weathercode": codes[i] if i < len(codes) else 0
            }
            hourly_forecast.append(hour_forecast) #Adds hour's forecast to the list
        return {
            "country": city_info["country"],
            "coordinate": {
                "latitude": city_info["lat"],
                "longitude": city_info["lon"]
            },
            "current_weather": current_weather,
            "hourly_forecast": hourly_forecast
        }
    except Exception as e:
        print(f"Error processing data: {e}")
        return None

def collect_all_weather_data():
    
    print("EU CAPITALS WEATHER DATA COLLECTION")
    all_weather = {} #Create empty dictionary to store all weather data
    total_cities = len(eu_capitals)
    
    for i, city in enumerate(eu_capitals, 1):
        city_name = city["city"]
        print(f"\n [{i}/{total_cities}] Fetching: {city_name}, {city['country']}") #[1/27] Fetching: Vienna, Austria
        raw_data = fetch_weather(city["lat"], city["lon"])
        
        #Check if fetch_weather returned data
        if raw_data:
            #Process the raw data into required format
            processed = process_city_weather(city, raw_data)
        
            #Check if processing was successful
            if processed:
                all_weather[city_name] = processed
                print(f"Success - {len(processed['hourly_forecast'])} hours of data")
            else:
                print(f"Process failed")
        else:
            print(f"Failed to fetch data")
        
        if i < total_cities:
            #Wait 0.75 seconds before making next API request
            time.sleep(0.75)
    return all_weather

#Save data to JSON file
def save_to_json(data, filename = "eu_weather_data.json"):
    if not data:
        print("\n No data found.")
        return False
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii = False) #convert Python to JSON an write to file indent=2 - formatting with 2space indentation

        print(f"Data saved to '{filename}'")
        print(f"Total cities: {len(data)}")
        return True

    except Exception as e:
        print(f"\n Error saving to JSON format: {e}")
        return False

def display_summary(data):

    if not data:
        return

    print("DATA SUMMARY")
    
    #Create empty list to store all temperatures
    temps = []
    for city_name, city_data in data.items():
        
        temp = city_data["current_weather"]["temperature"]
        if temp is not None:
            temps.append(temp)
    
    if temps:
        avg_temp = sum(temps) / len(temps) #Calculate average temperature
        hottest = max(data.items(), key=lambda x: x[1]["current_weather"]["temperature"]) #Find hottest city with max()
        coldest = min(data.items(), key=lambda x: x[1]["current_weather"]["temperature"]) #Find coldest city min()
        
        print(f"Average Temperature: {avg_temp:.1f}°C")
        print(f"Hottest: {hottest[0]} ({hottest[1]['current_weather']['temperature']}°C)")
        print(f"Coldest: {coldest[0]} ({coldest[1]['current_weather']['temperature']}°C)")

def main():
    print("\n Weather data collection started...")
    
    #Print estimated time (27 cities × 0.75 seconds ≈ 20 seconds)
    print("This will take about 20-25 seconds...\n")
    
    #Collect all weather data by calling collect_all_weather_data()
    weather_data = collect_all_weather_data()
    
    #Save collected data to JSON file
    save_to_json(weather_data)
    
    #Display summary of collected data
    display_summary(weather_data)
    
    #Show sample data for first city
    if weather_data:
        first_city = list(weather_data.keys())[0]
        print(f"\n Sample data for {first_city}:")
        print(f"Country: {weather_data[first_city]['country']}")
        print(f"Current: {weather_data[first_city]['current_weather']['temperature']}°C")
        print(f"Condition: {weather_data[first_city]['current_weather']['condition']}")
        print(f"Hourly forecasts: {len(weather_data[first_city]['hourly_forecast'])} hours")
    

if __name__ == "__main__":
    try:
        #This import is redundant but checks availability
        import requests
        main()

    except ImportError: #If this error occurs (requests library not installed)
        print("Error: 'requests' library not installed.")
        print("Install it with: pip install requests")
    


 Weather data collection started...
This will take about 20-25 seconds...

EU CAPITALS WEATHER DATA COLLECTION

 [1/27] Fetching: Vienna, Austria
Success - 24 hours of data

 [2/27] Fetching: Brussels, Belgium
Success - 24 hours of data

 [3/27] Fetching: Sofia, Bulgaria
Success - 24 hours of data

 [4/27] Fetching: Zagreb, Croatia
Success - 24 hours of data

 [5/27] Fetching: Nicosia, Cyprus
Success - 24 hours of data

 [6/27] Fetching: Prague, Czechia
Success - 24 hours of data

 [7/27] Fetching: Copenhagen, Denmark
Success - 24 hours of data

 [8/27] Fetching: Tallinn, Estonia
Success - 24 hours of data

 [9/27] Fetching: Helsinki, Finland
Success - 24 hours of data

 [10/27] Fetching: Paris, France
Success - 24 hours of data

 [11/27] Fetching: Berlin, Germany
Success - 24 hours of data

 [12/27] Fetching: Athens, Greece
Success - 24 hours of data

 [13/27] Fetching: Budapest, Hungary
Success - 24 hours of data

 [14/27] Fetching: Dublin, Ireland
Success - 24 hours of data

 [15/2